# DSCI 100 Group Project Report (Group 17)
## Factors of Patients in Heart Disease (Chosen Data Set: Heart Disease)

In [21]:
library(tidyverse)
library(tidymodels)

In [22]:
colnames <- c("id","ccf","age","sex","painloc","painexer","relrest","pncaden","cp","trestbps","htn","chol","smoke","cigs","years","fbs","dm","famhist","restecg","ekgmo","ekgday","ekgyr","dig","prop","nitr","pro","diuretic","proto","thaldur","thaltime","met","thalach","thalrest","tpeakbps","tpeakbpd","dummy","trestbpd","exang","xhypo","oldpeak","slope","rldv5","rldv5e","ca","restckm","exerckm","restef","restwm","exeref","exerwm","thal","thalsev","thalpul","earlobe","cmo","cday","cyr","num","lmt","ladprox","laddist","diag","cxmain","ramus","om1","om2","rcaprox","rcadist","lvx1","lvx2","lvx3","lvx4","lvf","cathef","junk","name")
data <- read_delim("https://raw.githubusercontent.com/alexcw-5/DSCI-100-Group-Project/master/cleveland.csv", delim = " ", col_names=colnames) %>%
        select(thalrest, cigs, years, tpeakbps, tpeakbpd, num)
data

Parsed with column specification:
cols(
  .default = col_double(),
  name = col_character()
)

See spec(...) for full column specifications.



thalrest,cigs,years,tpeakbps,tpeakbpd,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
60,50,20,190,90,0
64,40,40,160,90,2
78,20,35,140,80,1
84,0,0,195,68,0
71,0,0,160,74,0
73,20,20,165,70,0
83,0,0,180,84,3
84,0,0,165,80,0
75,0,0,120,70,2


### Introduction

#### Background Information

In the United States and Canada, heart disease is one of the leading causes of death among most age, gender, and racial groups. There are many lifestyle risk factors and contributors to heart disease in a person, and it is a growing issue that requires further awareness and analysis to tackle. In particular, risk of heart disease has been observed to increase with age and high cholesterol levels, due to fatty deposits that build up and cause blockages in major arteries. The classification question we would like to answer with our project is the following: Are the factors of age and cholesterol levels able to predict the presence of heart disease in a patient? We have chosen to use serum cholesterol results as one of our main predictors due to its ability to show a variety of heart related information, such as if the patient has had a previous heart attack and/or high blood pressure.

The ‘processed-cleveland.data’ dataset we will be using has data on the medical info of 303 patients, consisting of 14 attributes *(listed in appendix)

### Methodology

text here

### Results

text here

### Discussion

text here

***
## Appendix

#### Bibliography

* text here